In [1]:
import pickle
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.cross_validation import KFold, train_test_split
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.grid_search import GridSearchCV
from sklearn.datasets import load_iris, load_digits, load_boston
%matplotlib inline
import seaborn as sns
from sklearn.metrics import r2_score, mean_squared_error, make_scorer
from collections import OrderedDict
import gc
from sklearn import datasets, linear_model,preprocessing
from IPython.display import display, HTML
import time
import nltk.corpus
from nltk.stem.snowball import SnowballStemmer

In [2]:
def RMSE(y_true,y_pred):
    mse = np.sqrt(mean_squared_error(y_true, y_pred))
    print 'MSE: %2.3f' % mse
    return mse

In [3]:
#take 1 CSV, then split it to 3..
class FeatureEngineering:

    def __init__(self, ValidationStart, ValidationEnd, trainHdfPath, trainHdfFile, testHdfPath1, testHdfPath2, testHdfFile, 
                 testTypes, trainTypes, trainCsvPath, testCsvPath, maxLag=0):
        self.ValidationStart = ValidationStart
        self.ValidationEnd = ValidationEnd
        self.maxLag = maxLag
        self.trainHdfPath = trainHdfPath
        self.trainHdfFile = trainHdfFile
        self.testHdfPath1 = testHdfPath1
        self.testHdfPath2 = testHdfPath2
        self.testHdfFile = testHdfFile
        self.testTypes = testTypes
        self.trainTypes = trainTypes
        self.trainCsvPath = trainCsvPath
        self.testCsvPath = testCsvPath
        
    @staticmethod
    def __printDataFrameBasics__(data):
        display(data.head(2))
        #print data.dtypes
        gc.collect()
        print(data.info(memory_usage=True))
        
    @staticmethod    
    def changeIndexTypeToLowerMemory(data):
        ##########
        #This is very critical, i accept max number is 2^32. Also, if don't do that, memory gets so much higher..
        ##########
        #data.reset_index(inplace=True)
        #data.drop("index",axis=1, inplace=True)
        #data.index = data.index.astype('uint32')
        gc.collect()
        
    def ReadHdf(self, trainOrTestOrBoth):
        '''Reads and holds Df in object memory'''            
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train = pd.read_hdf(self.trainHdfPath,self.trainHdfFile)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
            
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test1 = pd.read_hdf(self.testHdfPath1,self.testHdfFile)
            self.test2 = pd.read_hdf(self.testHdfPath2,self.testHdfFile)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test1)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test2)
            FeatureEngineering.__printDataFrameBasics__(self.test1)
            FeatureEngineering.__printDataFrameBasics__(self.test2)
        
    def ReadCsv(self, trainOrTestOrBoth):
        '''Reads and holds Df in memory'''
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth == 'both':
            self.train = pd.read_csv(self.trainCsvPath, usecols=self.trainTypes.keys(), dtype=self.trainTypes)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            tempTest = pd.read_csv(self.testCsvPath, usecols=self.testTypes.keys(), dtype=self.testTypes)
            self.test1 = tempTest.loc[tempTest.Semana.values == self.ValidationStart]
            self.test2 = tempTest.loc[tempTest.Semana.values == self.ValidationEnd]
            del tempTest
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test1)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test2)
            FeatureEngineering.__printDataFrameBasics__(self.test1)
            FeatureEngineering.__printDataFrameBasics__(self.test2)
            
    @staticmethod
    def ConvertCsvToHdf(csvPath, HdfPath, HdfName, ColumnTypeDict ):
        tempDf = pd.read_csv(csvPath, usecols=ColumnTypeDict.keys(), dtype=ColumnTypeDict,index=False)
        tempDf.to_hdf(HdfPath, HdfName, format='t')
        del tempDf
        gc.collect()
        print "ConvertCsvToHdf is done.."

    def Preprocess(self, trainOrTestOrBoth, columnFunctionTypeList):
        '''columnFunctionTypeList = [ ['C1',Func1,Type], ['C2',Func2,Type],..    ]'''
        for column, func, localType in columnFunctionTypeList:
            if trainOrTestOrBoth == 'train' or trainOrTestOrBoth =='both':
                self.train.loc[:,column] =  np.apply_along_axis(func,0,FE.train[column].values).astype(localType)
                #np.apply_along_axis(lambda x: x+1,0,FE.train["Semana"]).astype("int32")
            if trainOrTestOrBoth == 'test' or trainOrTestOrBoth == 'both':
                self.test1.loc[:,column] =  np.apply_along_axis(func,0,FE.test1[column].values).astype(localType)
                self.test2.loc[:,column] =  np.apply_along_axis(func,0,FE.test2[column].values).astype(localType)
        gc.collect()
        
    def SaveDataFrameToHdf(self,trainOrTestOrBoth):
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train.to_hdf(self.trainHdfPath, self.trainHdfFile, format='t', index="False")
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test1.to_hdf(self.testHdfPath1, self.testHdfFile, format='t', index="False")
            self.test2.to_hdf(self.testHdfPath2, self.testHdfFile, format='t', index="False")
        
    def AddDemandaGeneralMean(self,trainOrTestOrBoth): 
        #self.train.loc[:,"DemandaGeneralMean"] = self.train["Demanda_uni_equil"].loc[
         #   self.train.loc[:,'Semana'] < 10].mean().astype("float32")
            
        meanOfDemanda = self.train["Demanda_uni_equil"].values.mean().astype("float32")
        
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train.loc[:,"DemandaGeneralMean"] = meanOfDemanda
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test1.loc[:,"DemandaGeneralMean"] = meanOfDemanda
            self.test2.loc[:,"DemandaGeneralMean"] = meanOfDemanda
        
        #self.train.loc[:,"DemandaGeneralMean"] = self.train["Demanda_uni_equil"].values[
        #(self.train.loc[:,'Semana'].values < self.ValidationStart).values].mean().astype("float32")
        gc.collect()
        
    '''ConfigElements(0,[ ("A",["Semana","Agencia_ID"],["count","count"]),'''
    def AddConfigurableFeaturesToTrain(self, config):
        if config.lag > self.maxLag:
            self.maxLag = config.lag
        
        tempData = self.train[self.train['Semana'].values <= (self.ValidationEnd - config.lag)]
        #display(tempData)
        if(config.lag != 0):
            tempData.loc[:,'Semana'] = tempData['Semana'].values + config.lag
        #display(tempData)
        
        #Means iterative.. eliminate as long as np.nan exists..If there is already one, don't create, use the existing
        if config.targetVariable != "" and  config.targetVariable not in self.train.columns:
            self.train.loc[:,config.targetVariable] = np.nan
            self.test1.loc[:,config.targetVariable] = np.nan
            
            if config.lag != 1:
                self.test2.loc[:,config.targetVariable] = np.nan
        
        for name,groups,aggregate in config.nameAndGroups:
            if name not in self.train.columns:
                print "{} is not in columns..".format(name)            
                
                groupedDataframe = tempData[groups+['Demanda_uni_equil']].copy().groupby(groups).agg(aggregate[0])
                gc.collect()
                #groupedDataframe.columns = groupedDataframe.columns.droplevel(0)
                groupedDataframe.columns = [name]
                
                #This is means of the counts of the semana-columns tuples!..!!!
                #If no lag and mean, mean of the columns without semana!!..
                #If there is lag and count, count of the columns x weeks before
                #If there is lag and mean, mean of the columns x weeks before
                #if(config.lag == 0 and aggregate == "count"):
                if(len(aggregate)>1):
                    groupedDataframe.reset_index(inplace=True)
                    groupedDataframe.drop("Semana",axis=1, inplace=True)
                    groups = groups[1:]
                    groupedDataframe = groupedDataframe.groupby(groups).agg(aggregate[1])
                    groupedDataframe.columns = [name]
                    gc.collect()
                
                display(groupedDataframe)
                self.train = self.train.merge( groupedDataframe, left_on=groups,
                    right_index=True, how='left', sort=False,copy=False)
                gc.collect()
                self.test1 = self.test1.merge( groupedDataframe, left_on=groups,
                    right_index=True, how='left', sort=False,copy=False)
                gc.collect()
                if config.lag != 1:
                    self.test2 = self.test2.merge( groupedDataframe, left_on=groups,
                        right_index=True, how='left', sort=False,copy=False)
                
                del groupedDataframe
                gc.collect()
            else:
                print "{} is in columns..".format(name)
            
            display(self.train)
            display(self.test1)
            display(self.test2)
            
            #Means iterative..!!!!!
            if config.targetVariable != "":
                self.train.loc[pd.isnull(self.train[config.targetVariable].values), 
                    config.targetVariable] = self.train.loc[pd.isnull(self.train[config.targetVariable].values)
                    , name].values
                self.test1.loc[pd.isnull(self.test1[config.targetVariable].values), 
                    config.targetVariable] = self.test1.loc[pd.isnull(self.test1[config.targetVariable].values),
                    name].values
                if config.lag != 1:
                    self.test2.loc[pd.isnull(self.test2[config.targetVariable].values), 
                        config.targetVariable] = self.test2.loc[pd.isnull(self.test2[config.targetVariable].values)
                        , name].values
                    
                count = self.test1[config.targetVariable].isnull().sum()
                print "Count of missing numbers after {} in validation part 1 in column {} is {}".format(name, 
                    config.targetVariable,str(count))
                if config.lag != 1:
                    count = self.test2.loc[:,config.targetVariable].isnull().sum()
                    print "Count of missing numbers after {} in validation part 2 in column {} is {}".format(name, 
                        config.targetVariable,str(count))
                
                
                #display(self.train)
                #If column is already in Dataframe and we want to fill target variable, this deletes columns!!!
                if(config.deleteColumns):
                    self.train.drop(name, axis=1, inplace=True)
                    self.test1.drop(name, axis=1, inplace=True)
                    if config.lag != 1:
                        self.test2.drop(name, axis=1, inplace=True)
                gc.collect()
                #Only in tesst
                #if count == 0:
                 #   break
        del tempData
        display(self.train)   
        display(self.test1)   
        display(self.test2)
        gc.collect()
        return 
    
    def DeleteLaggedWeeksFromTrain(self):
        self.train = self.train[self.train['Semana'].values >= (3 + self.maxLag)]
        gc.collect()
        display(self.train.head(2))
        
    def ReadFirstNRowsOfACsv(self, nrows, trainOrTestOrBoth) :
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train = pd.read_csv(self.trainCsvPath, usecols=self.trainTypes.keys(), dtype=self.trainTypes, nrows = nrows)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            tempTest = pd.read_csv(self.testCsvPath, usecols=self.testTypes.keys(), dtype=self.testTypes, nrows = nrows*2)
            self.test1 = tempTest.loc[tempTest.Semana == self.ValidationStart]
            self.test2 = tempTest.loc[tempTest.Semana == self.ValidationEnd]
            del tempTest
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test1)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test2)
            FeatureEngineering.__printDataFrameBasics__(self.test1)
            FeatureEngineering.__printDataFrameBasics__(self.test2)
    
    #Use when concatanating train and validation before predict test for example..
    def AppendTestToTrain(self,deleteTest = True):
        self.train = self.train.append(self.test1,ignore_index=True)
        gc.collect()
        if(deleteTest):
            del self.test1
            gc.collect()
        try:
            self.train = self.train.append(self.test2,ignore_index=True)
            gc.collect()
            if(deleteTest):
                del self.test2
                gc.collect()
        except:
            pass
        #BAD PERFORMANCE!!
    #Split train data to train and test1 and test2 (validation)
    #def SplitTrainToTestUsingValidationStart(self):
     #   boolCondition = self.train.Semana == self.ValidationStart
      #  self.test1 = self.train.loc[boolCondition]
       # self.train.drop((self.train.loc[boolCondition].index), axis=0,inplace=True)
        
       # boolCondition = self.train.Semana == self.ValidationEnd
       # self.test2 = self.train.loc[boolCondition]
       # self.train.drop((self.train.loc[boolCondition].index), axis=0,inplace=True)
      #  del boolCondition
      #  gc.collect()
    
    #Reaches 3x memory from train, because of test1, test2 and train itself at the end.. GC fixed in the end..
    def SplitTrainToTestUsingValidationStart(self):
        boolCondition = self.train.Semana.values == self.ValidationStart
        self.test1 = self.train[boolCondition]
        boolCondition = self.train.Semana.values == self.ValidationEnd
        self.test2 = self.train[boolCondition]
        FE.train = FE.train[ FE.train.Semana.values < FE.ValidationStart ]
        del boolCondition
        gc.collect()
        
    def XgboostPredictAndSee(self, Test1OrTest2):
        self.train_y = self.train["Demanda_uni_equil"].copy()
        self.train.drop("Demanda_uni_equil",axis=1, inplace=True)
        

In [4]:
parameterDict =       {"ValidationStart":8, 
 "ValidationEnd":9,
   "maxLag":3,
    "trainHdfPath":'../../input/train.h5',
    "trainHdfFile":"train",
    "testHdfPath1":"../../input/test1.h5",
    "testHdfPath2":"../../input/test2.h5",
    "testHdfFile":"test", 
    "trainTypes" : {'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16, 
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16,'Venta_uni_hoy':np.uint16, 'Venta_hoy':np.float32,
                    'Dev_uni_proxima': np.uint32, 'Dev_proxima':np.float32,'Demanda_uni_equil':np.uint32}, 
    "testTypes" : {'id':np.uint32,'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16,
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16},
    "trainCsvPath":'../../input/train.csv'   ,
    "testCsvPath":'../../input/test.csv'}

FE = FeatureEngineering(**parameterDict)
print FE.__dict__

{'trainCsvPath': '../../input/train.csv', 'maxLag': 3, 'testTypes': {'Cliente_ID': <type 'numpy.uint32'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Semana': <type 'numpy.uint8'>, 'id': <type 'numpy.uint32'>}, 'testHdfFile': 'test', 'trainTypes': {'Dev_proxima': <type 'numpy.float32'>, 'Venta_uni_hoy': <type 'numpy.uint16'>, 'Cliente_ID': <type 'numpy.uint32'>, 'Demanda_uni_equil': <type 'numpy.uint32'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Venta_hoy': <type 'numpy.float32'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Dev_uni_proxima': <type 'numpy.uint32'>, 'Semana': <type 'numpy.uint8'>}, 'testHdfPath1': '../../input/test1.h5', 'ValidationEnd': 9, 'testHdfPath2': '../../input/test2.h5', 'testCsvPath': '../../input/test.csv', 'ValidationStart': 8, 'trainHdfFile': 'train', 'trainHdfPath': '../../input/train.h5'}


In [5]:
FE.ReadHdf('both')

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,Town_ID,State_ID,DemandaGeneralMean,Lag0,Lag0Averaged,Demanda_uni_equilLogged,Lag0AveragedLogged,Lag1,Lag2,Lag3
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,...,2008,16,7.346896,12.0,2.0,1.386294,1.098612,NaN,NaN,NaN
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,...,2008,16,7.346896,12.0,2.0,1.609438,1.098612,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
Int64Index: 52449630 entries, 0 to 52449629
Data columns (total 26 columns):
Semana                                          uint8
Agencia_ID                                      uint16
Canal_ID                                        uint8
Ruta_SAK                                        uint16
Cliente_ID                                      uint32
Producto_ID                                     uint16
Venta_uni_hoy                                   uint16
Venta_hoy                                       float32
Dev_uni_proxima                                 uint32
Dev_proxima                                     float32
Demanda_uni_equil                               uint32
DemandaNotEqualTheDifferenceOfVentaUniAndDev    bool
weight                                          uint16
pieces                                          uint8
Prod_name_ID                                    uint16
Brand_ID                                        uint8
Town_ID  

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,Town_ID,State_ID,DemandaGeneralMean,Lag0,Lag0Averaged,Demanda_uni_equilLogged,Lag0AveragedLogged,Lag1,Lag2,Lag3
52449630,8,1110,7,3301,15766,1212,4,33.520000,0,0.0,...,2008,16,7.346896,12.0,2.0,1.609438,1.098612,5.864407,5.931034,5.0
52449631,8,1110,7,3301,15766,1216,5,41.900002,0,0.0,...,2008,16,7.346896,12.0,2.0,1.791759,1.098612,2.000000,1.000000,3.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10193576 entries, 52449630 to 62643205
Data columns (total 26 columns):
Semana                                          uint8
Agencia_ID                                      uint16
Canal_ID                                        uint8
Ruta_SAK                                        uint16
Cliente_ID                                      uint32
Producto_ID                                     uint16
Venta_uni_hoy                                   uint16
Venta_hoy                                       float32
Dev_uni_proxima                                 uint32
Dev_proxima                                     float32
Demanda_uni_equil                               uint32
DemandaNotEqualTheDifferenceOfVentaUniAndDev    bool
weight                                          uint16
pieces                                          uint8
Prod_name_ID                                    uint16
Brand_ID                                        uint8
To

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,Brand_ID,Town_ID,State_ID,DemandaGeneralMean,Lag0,Lag0Averaged,Demanda_uni_equilLogged,Lag0AveragedLogged,Lag2,Lag3
62643206,9,1110,7,3301,15766,1212,1,8.38,0,0.0,...,4,2008,16,7.346896,12.0,2.0,0.693147,1.098612,5.864407,5.931034
62643207,9,1110,7,3301,15766,1238,2,19.66,0,0.0,...,4,2008,16,7.346896,12.0,2.0,1.098612,1.098612,2.000000,2.000000


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10200437 entries, 62643206 to 72843642
Data columns (total 25 columns):
Semana                                          uint8
Agencia_ID                                      uint16
Canal_ID                                        uint8
Ruta_SAK                                        uint16
Cliente_ID                                      uint32
Producto_ID                                     uint16
Venta_uni_hoy                                   uint16
Venta_hoy                                       float32
Dev_uni_proxima                                 uint32
Dev_proxima                                     float32
Demanda_uni_equil                               uint32
DemandaNotEqualTheDifferenceOfVentaUniAndDev    bool
weight                                          uint16
pieces                                          uint8
Prod_name_ID                                    uint16
Brand_ID                                        uint8
To

In [6]:
FE.DeleteLaggedWeeksFromTrain()

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,Town_ID,State_ID,DemandaGeneralMean,Lag0,Lag0Averaged,Demanda_uni_equilLogged,Lag0AveragedLogged,Lag1,Lag2,Lag3
32263831,6,1110,7,3301,15766,1216,1,8.38,0,0.0,...,2008,16,7.346896,12.0,2.0,0.693147,1.098612,3.0,2.0,4.0
32263832,6,1110,7,3301,15766,1238,2,19.66,0,0.0,...,2008,16,7.346896,12.0,2.0,1.098612,1.098612,1.0,3.0,4.0


In [7]:
FE.train.columns

Index([u'Semana', u'Agencia_ID', u'Canal_ID', u'Ruta_SAK', u'Cliente_ID',
       u'Producto_ID', u'Venta_uni_hoy', u'Venta_hoy', u'Dev_uni_proxima',
       u'Dev_proxima', u'Demanda_uni_equil',
       u'DemandaNotEqualTheDifferenceOfVentaUniAndDev', u'weight', u'pieces',
       u'Prod_name_ID', u'Brand_ID', u'Town_ID', u'State_ID',
       u'DemandaGeneralMean', u'Lag0', u'Lag0Averaged',
       u'Demanda_uni_equilLogged', u'Lag0AveragedLogged', u'Lag1', u'Lag2',
       u'Lag3'],
      dtype='object')

In [8]:
FE.test1.columns

Index([u'Semana', u'Agencia_ID', u'Canal_ID', u'Ruta_SAK', u'Cliente_ID',
       u'Producto_ID', u'Venta_uni_hoy', u'Venta_hoy', u'Dev_uni_proxima',
       u'Dev_proxima', u'Demanda_uni_equil',
       u'DemandaNotEqualTheDifferenceOfVentaUniAndDev', u'weight', u'pieces',
       u'Prod_name_ID', u'Brand_ID', u'Town_ID', u'State_ID',
       u'DemandaGeneralMean', u'Lag0', u'Lag0Averaged',
       u'Demanda_uni_equilLogged', u'Lag0AveragedLogged', u'Lag1', u'Lag2',
       u'Lag3'],
      dtype='object')

In [9]:
FE.test1.head(2)

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,Town_ID,State_ID,DemandaGeneralMean,Lag0,Lag0Averaged,Demanda_uni_equilLogged,Lag0AveragedLogged,Lag1,Lag2,Lag3
52449630,8,1110,7,3301,15766,1212,4,33.520000,0,0.0,...,2008,16,7.346896,12.0,2.0,1.609438,1.098612,5.864407,5.931034,5.0
52449631,8,1110,7,3301,15766,1216,5,41.900002,0,0.0,...,2008,16,7.346896,12.0,2.0,1.791759,1.098612,2.000000,1.000000,3.0


In [10]:
X_train = FE.train[["Lag0AveragedLogged","Lag1","Lag2","Lag3","weight","pieces"]]
X_test = FE.test1[["Lag0AveragedLogged","Lag1","Lag2","Lag3","weight","pieces"]]
y_train = FE.train["Demanda_uni_equilLogged"]
y_test = FE.test1["Demanda_uni_equilLogged"]

In [11]:
gc.collect()

613

In [12]:
print X_train.shape
print X_test.shape
print y_train.shape
print y_test.shape

(20185799, 6)
(10193576, 6)
(20185799,)
(10193576,)


In [ ]:
Demanda_uni_equilLogged	Lag0AveragedLogged	Lag1	Lag2	Lag3
weight	pieces	Brand_ID	State_ID	


In [13]:
display(X_train[:3])
display(X_test[:3])
display(y_train[:3])
display(y_test[:3])

,Lag0AveragedLogged,Lag1,Lag2,Lag3,weight,pieces
32263831,1.098612,3.0,2.0,4.0,135,2
32263832,1.098612,1.0,3.0,4.0,140,2
32263833,1.386294,2.0,3.0,3.0,105,6


,Lag0AveragedLogged,Lag1,Lag2,Lag3,weight,pieces
52449630,1.098612,5.864407,5.931034,5.0,120,2
52449631,1.098612,2.000000,1.000000,3.0,135,2
52449632,0.693147,4.833333,6.040816,3.0,130,6


32263831    0.693147
32263832    1.098612
32263833    1.386294
Name: Demanda_uni_equilLogged, dtype: float64

52449630    1.609438
52449631    1.791759
52449632    0.693147
Name: Demanda_uni_equilLogged, dtype: float64

In [14]:
defaultParams = {"max_depth":6, "subsample":1., "min_child_weight":5, "colsample_bytree":0.4, "missing":np.nan
                ,"n_estimators":500, "learning_rate":0.1}
xgb_model = xgb.XGBRegressor(**defaultParams) 


xgb_model.fit(X_train,y_train,eval_set=[(X_train, y_train),(X_test, y_test)],
         verbose=1, early_stopping_rounds = 50)


predictions = xgb_model.predict(X_train,ntree_limit  = xgb_model.best_iteration+1)
predictionsTest = xgb_model.predict(X_test,ntree_limit  = xgb_model.best_iteration+1)

print "trainResult: ", np.sqrt(mean_squared_error(y_train, predictions))
print "testResult: ", np.sqrt(mean_squared_error(y_test, predictionsTest))
print "Best Iteration: ", xgb_model.best_iteration
print "Best Score: ", xgb_model.best_score
print xgb.plot_importance(xgb_model)

[0]	validation_0-rmse:1.2713	validation_1-rmse:1.27229
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 50 rounds.
[1]	validation_0-rmse:1.15657	validation_1-rmse:1.16118
[2]	validation_0-rmse:1.05532	validation_1-rmse:1.06333
[3]	validation_0-rmse:0.970117	validation_1-rmse:0.978906
[4]	validation_0-rmse:0.889093	validation_1-rmse:0.902629
[5]	validation_0-rmse:0.81821	validation_1-rmse:0.836744
[6]	validation_0-rmse:0.759621	validation_1-rmse:0.780018
[7]	validation_0-rmse:0.707308	validation_1-rmse:0.729417
[8]	validation_0-rmse:0.657692	validation_1-rmse:0.686465
[9]	validation_0-rmse:0.618945	validation_1-rmse:0.650528
[10]	validation_0-rmse:0.58478	validation_1-rmse:0.619505
[11]	validation_0-rmse:0.555548	validation_1-rmse:0.593215
[12]	validation_0-rmse:0.526563	validation_1-rmse:0.571272
[13]	validation_0-rmse:0.508494	validation_1-rmse:0.555787
[14]	validation_0-rmse:0.490263	va

KeyboardInterrupt: 

In [6]:
def GiveBestParameterWithoutCV(defaultParams, testParams, X_train, X_test, Y_train, Y_test, fitParams):
    xgb_model = xgb.XGBRegressor(**defaultParams) 
    
    minRmse = 10000
    minRmseParameter = 10000
    bestIteration = 1000
        
    for key,values in testParams:
        minRmseParameter = xgb_model.get_xgb_params()[key]
        
        for value in values:
            
            xgb_model.set_params(**{key:value})
            xgb_model.fit(X_train,Y_train, eval_set=[(X_train, Y_train),(X_test, Y_test)],
                  **fitParams)
            if xgb_model.best_score < minRmse:
                minRmse = xgb_model.best_score
                minRmseParameter = value
                bestIteration = xgb_model.best_iteration
                
        xgb_model.set_params(**{key:minRmseParameter})
        print "Parameters are finished for {}. Best Iteration is {}".format(key, bestIteration)
        print "Minimum Rmse : {}, optimum parameter is {} between {}".format(minRmse, minRmseParameter, values)
    gc.collect()

In [7]:
defaultParams = {"max_depth":10, "subsample":1., "min_child_weight":5, "colsample_bytree":0.4, "missing":np.nan
                ,"n_estimators":300, "learning_rate":0.1}
testParams = [("max_depth",[12,8,6,14]), ("subsample",[0.9,0.8,0.6]), ("min_child_weight",[1,3,7]),
                  ("colsample_bytree",[0.3,0.5,0.6, 0.8,1]), ("learning_rate",[0.05])]
fitParams = {"verbose":10, "early_stopping_rounds": 50}

GiveBestParameterWithoutCV(defaultParams,testParams, X_train, X_test, y_train, y_test, fitParams )

[0]	validation_0-rmse:23.1009	validation_1-rmse:14.6234
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:9.25949	validation_1-rmse:4.56019
[20]	validation_0-rmse:4.13939	validation_1-rmse:3.47237
[30]	validation_0-rmse:2.25765	validation_1-rmse:4.62288
[40]	validation_0-rmse:1.48033	validation_1-rmse:5.12618
[50]	validation_0-rmse:1.05455	validation_1-rmse:5.39472
[60]	validation_0-rmse:0.818546	validation_1-rmse:5.48539
Stopping. Best iteration:
[17]	validation_0-rmse:5.13125	validation_1-rmse:3.37505

[0]	validation_0-rmse:23.1009	validation_1-rmse:14.6234
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:9.26368	validation_1-rmse:4.569
[20]	validation_0-rmse:4.15646	validation_1-rmse:3.53157
[30]	validation_0-rmse:2.3231	v